In [1]:
import sys
sys.path.insert(0,'../')

In [2]:
import yaml
import open3d as o3d
import numpy as np
from data.WaymoDataset import WaymoDataset
from data.util import ApplyPillarization, drop_points_function
from utils import str2bool
from visualization.util import predict_and_store_flows, flows_exist
from models.FastFlow3DModelScatter import FastFlow3DModelScatter

from data.preprocess import points_in_boxes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
data_dir = "/media/robot/hdd/waymo_open_dataset_scene_flow/train_prep"

In [4]:
waymo_dataset = WaymoDataset(data_dir)

In [5]:
waymo_dataset.return_boxes = True

In [6]:
config_info = {}
config_info["grid_cell_size"] = {"value": 0.33203125}
config_info["x_min"] = {"value": -85}
config_info["x_max"] = {"value": 85}
config_info["y_min"] = {"value": -85}
config_info["y_max"] = {"value": 85}
config_info["z_min"] = {"value": -3}
config_info["z_max"] = {"value": 3}
config_info["n_pillars_x"] = {"value": 512}
config_info["n_pillars_y"] = {"value": 512}

In [7]:
def text_3d(text, pos, direction=None, degree=0.0, font='DejaVu Sans Mono for Powerline.ttf', font_size=16):
    """
    Generate a 3D text point cloud used for visualization.
    :param text: content of the text
    :param pos: 3D xyz position of the text upper left corner
    :param direction: 3D normalized direction of where the text faces
    :param degree: in plane rotation of text
    :param font: Name of the font - change it according to your system
    :param font_size: size of the font
    :return: o3d.geoemtry.PointCloud object
    """
    if direction is None:
        direction = (0., 0., 1.)

    from PIL import Image, ImageFont, ImageDraw
    from pyquaternion import Quaternion
#     ImageFont.truetype(, 15)
    font_obj = ImageFont.truetype("arial.ttf", font_size)
    font_dim = font_obj.getsize(text)

    img = Image.new('RGB', font_dim, color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 0), text, font=font_obj, fill=(0, 0, 0))
    img = np.asarray(img)
    img_mask = img[:, :, 0] < 128
    indices = np.indices([*img.shape[0:2], 1])[:, img_mask, 0].reshape(3, -1).T

    pcd = o3d.geometry.PointCloud()
    pcd.colors = o3d.utility.Vector3dVector(img[img_mask, :].astype(float) / 255.0)
    pcd.points = o3d.utility.Vector3dVector(indices / 100.0)

    raxis = np.cross([0.0, 0.0, 1.0], direction)
    if np.linalg.norm(raxis) < 1e-6:
        raxis = (0.0, 0.0, 1.0)
    trans = (Quaternion(axis=raxis, radians=np.arccos(direction[2])) *
             Quaternion(axis=direction, degrees=degree)).transformation_matrix
    trans[0:3, 3] = np.asarray(pos)
    pcd.transform(trans)
    return pcd

In [8]:
# grid_cell_size = config_info['grid_cell_size']['value']
# x_min = config_info['x_min']['value']
# y_min = config_info['y_min']['value']
# z_min = config_info['z_min']['value']
# x_max = config_info['x_max']['value']
# y_max = config_info['y_max']['value']
# z_max = config_info['z_max']['value']
# n_pillars_x = config_info['n_pillars_x']['value']
# n_pillars_y = config_info['n_pillars_y']['value']
# point_cloud_transform = ApplyPillarization(grid_cell_size=grid_cell_size, x_min=x_min,
#                                            y_min=y_min, z_min=z_min, z_max=z_max, n_pillars_x=n_pillars_x)

# waymo_dataset.set_point_cloud_transform(point_cloud_transform)
# drop_points_function = drop_points_function(x_min=x_min,
#                                             x_max=x_max, y_min=y_min, y_max=y_max,
#                                             z_min=z_min, z_max=z_max)
# waymo_dataset.set_drop_invalid_point_function(drop_points_function)

### Load data

In [9]:
data = waymo_dataset[100]

In [10]:
(previous_frame, current_frame), flows, current_bbox, previous_bbox, current_ind, previous_ind = data

In [11]:
prev_points = previous_frame[0][:, 0:3]
cur_points = current_frame[0][:, 0:3]

### Create lables (bbox ids) to be visualized

In [12]:

labels = []
for label, bbox in zip(current_ind, current_bbox):
    labels.append(text_3d(label, bbox[0]))

    

### Create o3d bboxes to be visualized

In [13]:
line_sets = []
lines = [[0, 1], [1, 2], [2, 3], [0, 3],
                 [4, 5], [5, 6], [6, 7], [4, 7],
                 [0, 4], [1, 5], [2, 6], [3, 7]]

# Use the same color for all lines
colors = [[0, 1, 0] for _ in range(len(lines))]

for corner_box in current_bbox:
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(corner_box)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    line_sets.append(line_set)

### Get pointClouds only inside bboxes

In [14]:
cutted_points = cur_points[:, :3][points_in_boxes(current_bbox, cur_points)]

### Visualize

In [15]:
pcd_bboxes = o3d.geometry.PointCloud()
pcd_bboxes.points = o3d.utility.Vector3dVector(cutted_points)
pcd_bboxes.paint_uniform_color([0.1, 0.1, 0.7])
full_pc =  o3d.geometry.PointCloud()
full_pc.points = o3d.utility.Vector3dVector(cur_points)
full_pc.paint_uniform_color([0.7, 0.1, 0.1])
o3d.visualization.draw_geometries([full_pc, *labels, *line_sets, pcd_bboxes,])